# Testing APIs

#### Config

In [60]:
import configparser

# Load configuration
config = configparser.RawConfigParser()
config.read('../config/config.ini')

# Twitter API credentials
API_KEY = config['TWITTER_API']['API_KEY']
API_SECRET = config['TWITTER_API']['API_SECRET']
ACCESS_TOKEN = config['TWITTER_API']['ACCESS_TOKEN']
ACCESS_SECRET = config['TWITTER_API']['ACCESS_SECRET']
BEARER_TOKEN = config['TWITTER_API']['BEARER_TOKEN']

# Azure Event Hub Kafka settings
KAFKA_BROKER = config['AZURE_EVENTHUB']['BROKER']
EVENT_HUB_NAME = config['AZURE_EVENTHUB']['EVENT_HUB_NAME']
KAFKA_SASL_USERNAME = config['AZURE_EVENTHUB']['SASL_USERNAME']
KAFKA_SASL_PASSWORD = config['AZURE_EVENTHUB']['SASL_PASSWORD']

# Reddit API Settings
REDDIT_CLIENT_ID = config['REDDIT_API']['CLIENT_ID']
REDDIT_CLIENT_SECRET = config['REDDIT_API']['CLIENT_SECRET']
REDDIT_USER_AGENT = config['REDDIT_API']['USER_AGENT']
REDDIT_USERNAME = config['REDDIT_API']['USERNAME']
REDDIT_PASSWORD = config['REDDIT_API']['PASSWORD']

print("Config loaded successfully!")

Config loaded successfully!


#### Testing Tweepy

In [61]:
import tweepy
import pandas as pd
import time

In [47]:
auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True)

# Test authentication
try:
    api.verify_credentials()
    print("Authentication successful!")
except Exception as e:
    print(f"Error: {e}")


Authentication successful!


In [48]:
client = tweepy.Client(bearer_token=BEARER_TOKEN, consumer_key=API_KEY, consumer_secret=API_SECRET, access_token=ACCESS_TOKEN, access_token_secret=ACCESS_SECRET)
auth = tweepy.OAuth2BearerHandler(BEARER_TOKEN)
api = tweepy.API(auth)
# Search for recent tweets
query = "fashion -is:retweet lang:en"  # Keywords + filter for retweets
response = client.search_recent_tweets(query=query, max_results=10, tweet_fields=["created_at", "text"])

# Parse and store data
data = []
if response.data:
    for tweet in response.data:
        data.append({
            "id": tweet.id,
            "created_at": tweet.created_at,
            "text": tweet.text
        })

# Convert to DataFrame
df = pd.DataFrame(data)
print(df)


                    id                created_at  \
0  1869087265127125142 2024-12-17 18:28:24+00:00   
1  1869087235427172369 2024-12-17 18:28:17+00:00   
2  1869087207174328518 2024-12-17 18:28:10+00:00   
3  1869087182184689829 2024-12-17 18:28:04+00:00   
4  1869087157920600371 2024-12-17 18:27:59+00:00   
5  1869087150886867395 2024-12-17 18:27:57+00:00   
6  1869087145358712856 2024-12-17 18:27:56+00:00   
7  1869087143500648723 2024-12-17 18:27:55+00:00   
8  1869087122965381609 2024-12-17 18:27:50+00:00   
9  1869087119135895917 2024-12-17 18:27:49+00:00   

                                                text  
0  Checkout on this shop on Shopee! Fashion  http...  
1  @Spread_Star @voguebusiness @Khulood_Almani @S...  
2  I paid for express shipping, and Fashion Nova ...  
3  @Ashy_slashee Hmmm... 🤔\n\nIt's a tough one th...  
4  🌟HOT HOT earn more with our crypto pump! $BTC ...  
5  they're stronger than me because if i had the ...  
6  i keep a copy of K&amp;R C in russian a

In [49]:
df['text'][0]

'Checkout on this shop on Shopee! Fashion  https://t.co/XLTOcf7ygW: https://t.co/sygAl8rFQD'

#### Testing Reddit API

In [66]:
import praw
import pandas as pd

# Reddit API credentials
reddit = praw.Reddit(
    client_id=REDDIT_CLIENT_ID,
    client_secret=REDDIT_CLIENT_SECRET,
    user_agent=REDDIT_USER_AGENT,
    redirect_uri="http://localhost",
)

In [71]:
# Subreddit to analyze
subreddit = reddit.subreddit("malefashionadvice")

# Fetch posts from 'hot', 'new', or 'top'
posts = []
for post in subreddit.hot(limit=100):  # Change limit as needed
    posts.append({
        "title": post.title,
        "score": post.score,
        "num_comments": post.num_comments,
        "created_utc": post.created_utc,
        "selftext": post.selftext
    })

# Convert to DataFrame
df = pd.DataFrame(posts)
df.head()

,title,score,num_comments,created_utc,selftext
0,WAYWT (What Are You Wearing This Week?) - 23 O...,6,23,1.729688e+09,WAYWT = What Are You Wearing This Week (or a d...
1,➡️ Daily Simple Questions ⬅️- Style feedback a...,1,4,1.734455e+09,Welcome to the Daily Questions thread for all ...
2,I've held my tongue as long as I can.,405,125,1.734420e+09,Some of the questions that get asked here are ...
3,Does anyone know what this suit jacket style i...,2,0,1.734460e+09,[https://imgur.com/a/9SDo8Jw](https://imgur.co...
4,Tom Ford Oud Wood - 1.7oz,2,0,1.734460e+09,Tom Ford Oud Wood - 1.7oz for $110 from Costco...


In [77]:
df['selftext'][99]

'Looking to get my first proper (ie. not “smart”) watch for nicer outfits. Being both a Swiss design and rail travel aficionado, a Mondaine Swiss Rail clock watch seems perfect for my taste. Any opinions on them? Good quality for the price? Long lasting?'